# 시장데이터 종가 생성

1. col_62 파일 기반으로 재생성
2. 9920종가 파일 기반 - 99년 데이터부터 사용함으로서 정확도를 높임


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm 

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
plt.rcParams['font.family'] = 'malgun gothic'

from sklearn.metrics import accuracy_score, log_loss, plot_confusion_matrix, recall_score, precision_score, f1_score, roc_auc_score,average_precision_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier, Pool  
from lightgbm import LGBMClassifier   

from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler

import warnings
warnings.filterwarnings(action='ignore')

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## 데이터 불러오기

In [2]:
price = pd.read_csv(r'..\..\..\..\Code\SH\csv\9920종가.csv', encoding= 'euc-kr')
price_original = price

In [3]:
price

,회사명,거래소코드,회계년도,종가
0,(주)CMG제약,58820,2000/12,NaN
1,(주)CMG제약,58820,2001/12,1420.0
2,(주)CMG제약,58820,2002/12,520.0
3,(주)CMG제약,58820,2003/12,800.0
4,(주)CMG제약,58820,2004/12,550.0
...,...,...,...,...
27936,흥구석유(주),24060,2020/12,7220.0
27937,흥구석유(주),24060,2000/12,265000.0
27938,흥구석유(주),24060,2011/12,1500.0
27939,흥구석유(주),24060,2012/12,1585.0


In [4]:
price.isna().sum()

회사명         0
거래소코드       0
회계년도        0
종가       6807
dtype: int64

In [6]:
#일단 결측치 fillna로 0채워줌
#price = price.fillna(0)

## key 생성
거래소코드+년도

In [5]:
price['회계년도'].str[:4]

0        2000
1        2001
2        2002
3        2003
4        2004
         ... 
27936    2020
27937    2000
27938    2011
27939    2012
27940    2008
Name: 회계년도, Length: 27941, dtype: object

In [74]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27941 entries, 0 to 27940
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회사명     27941 non-null  object 
 1   거래소코드   27941 non-null  int64  
 2   회계년도    27941 non-null  object 
 3   종가      21134 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 873.3+ KB


In [6]:
price['년'] = price['회계년도'].str[:4].astype(int)

In [7]:
price['년'] = price['회계년도'].str[:4].astype(int)
price['key'] = price['거래소코드'].astype(str) + (price['년']+1).astype(str)

In [8]:
price

,회사명,거래소코드,회계년도,종가,년,key
0,(주)CMG제약,58820,2000/12,NaN,2000,588202001
1,(주)CMG제약,58820,2001/12,1420.0,2001,588202002
2,(주)CMG제약,58820,2002/12,520.0,2002,588202003
3,(주)CMG제약,58820,2003/12,800.0,2003,588202004
4,(주)CMG제약,58820,2004/12,550.0,2004,588202005
...,...,...,...,...,...,...
27936,흥구석유(주),24060,2020/12,7220.0,2020,240602021
27937,흥구석유(주),24060,2000/12,265000.0,2000,240602001
27938,흥구석유(주),24060,2011/12,1500.0,2011,240602012
27939,흥구석유(주),24060,2012/12,1585.0,2012,240602013


## 종가변동률 구하기

In [11]:
#종가변동률 구하는 식 <br>
#(df['자기자본배율'] - df['자기자본배율'].shift(1))/df['자기자본배율'].shift(1)*100

In [9]:
# for문으로 생성해주기
#종가변동률 구하는 식 <br>
#(df['자기자본배율'] - df['자기자본배율'].shift(1))/df['자기자본배율'].shift(1)*100
company = price['회사명'].unique().tolist()

for i in tqdm(range(len(company))):
    price.loc[price['회사명']==company[i], '종가변동률']=(price.loc[price['회사명']==company[i]]['종가'] - price.loc[price['회사명']==company[i]]['종가'].shift(1))/price.loc[price['회사명']==company[i]]['종가'].shift(1)
    #price = price.fillna(0)
    price['종가변동률'] = price['종가변동률'].replace([np.inf], np.nan)
    price['종가변동률'] = price['종가변동률'].fillna(method='bfill')# 후년 데이터로 대체
    

100%|██████████| 2051/2051 [00:14<00:00, 142.14it/s]


In [10]:
price

,회사명,거래소코드,회계년도,종가,년,key,종가변동률
0,(주)CMG제약,58820,2000/12,NaN,2000,588202001,-0.633803
1,(주)CMG제약,58820,2001/12,1420.0,2001,588202002,-0.633803
2,(주)CMG제약,58820,2002/12,520.0,2002,588202003,-0.633803
3,(주)CMG제약,58820,2003/12,800.0,2003,588202004,0.538462
4,(주)CMG제약,58820,2004/12,550.0,2004,588202005,-0.312500
...,...,...,...,...,...,...,...
27936,흥구석유(주),24060,2020/12,7220.0,2020,240602021,0.347015
27937,흥구석유(주),24060,2000/12,265000.0,2000,240602001,35.703601
27938,흥구석유(주),24060,2011/12,1500.0,2011,240602012,-0.994340
27939,흥구석유(주),24060,2012/12,1585.0,2012,240602013,0.056667


In [12]:
price.loc[price['회사명']==company[10]]

,회사명,거래소코드,회계년도,종가,년,key,종가변동률
130,(주)고영테크놀러지,98460,2005/12,NaN,2005,984602006,0.530758
131,(주)고영테크놀러지,98460,2006/12,NaN,2006,984602007,0.530758
132,(주)고영테크놀러지,98460,2007/12,NaN,2007,984602008,0.530758
133,(주)고영테크놀러지,98460,2008/12,6990.0,2008,984602009,0.530758
134,(주)고영테크놀러지,98460,2009/12,10700.0,2009,984602010,0.530758
135,(주)고영테크놀러지,98460,2010/12,18000.0,2010,984602011,0.682243
136,(주)고영테크놀러지,98460,2011/12,30900.0,2011,984602012,0.716667
137,(주)고영테크놀러지,98460,2012/12,28100.0,2012,984602013,-0.090615
138,(주)고영테크놀러지,98460,2013/12,27950.0,2013,984602014,-0.005338
139,(주)고영테크놀러지,98460,2014/12,42800.0,2014,984602015,0.531306


In [13]:
# 결측치 확인
price.isna().sum()

회사명         0
거래소코드       0
회계년도        0
종가       6807
년           0
key         0
종가변동률       0
dtype: int64

In [14]:
#csv저장
price.to_csv('9920종가변동률 반영.csv')

## finaldataset과 merge

In [2]:
#price = pd.read_csv(r'..\..\..\Code\SH\4. 텍스트마이닝\0020종가변동률 반영.csv')

In [17]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드', '상장폐지일', '감사의견코드', '자기자본배당률',
       '금융비용 대 부채비율', '자기자본배율', '유동부채비율', '매입채무회전률', '유동자산회전률', '총자본투자효율',
       '순운전자본비율', '누적수익성비율', '총자산영업이익률', '총자산회전율', '년', 'key', '부도', '공시발생일',
       '정기공시제목', '수시공시제목', '대표이사 변경', '최대주주 변경', '회계처리위반', '횡령배임', '신종채권',
       '영업조업중단', '출자목적_투자', '출자목적_경영권', '출자목적_영업이익', '기타'],
      dtype='object')

In [15]:
df_ = pd.read_csv(r'..\..\..\..\Code\SH\1. 데이터생성\최종 non0\col_62_.csv', index_col=0)
df = df_

In [16]:
df.shape

(27357, 68)

In [17]:
df['key']

0        588202001
1        588202002
2        588202003
3        588202004
4        588202005
           ...    
27352    240602017
27353    240602018
27354    240602019
27355    240602020
27356    240602021
Name: key, Length: 27357, dtype: int64

In [85]:
price['key']

0        588202001
1        588202002
2        588202003
3        588202004
4        588202005
           ...    
27936    240602021
27937    240602001
27938    240602012
27939    240602013
27940    240602009
Name: key, Length: 27941, dtype: object

In [86]:
price[['key','종가변동률']]

,key,종가변동률
0,588202001,-0.633803
1,588202002,-0.633803
2,588202003,-0.633803
3,588202004,0.538462
4,588202005,-0.312500
...,...,...
27936,240602021,0.347015
27937,240602001,35.703601
27938,240602012,-0.994340
27939,240602013,0.056667


In [18]:
df['key'] = df['key'].astype(str)

In [19]:
df['key']

0        588202001
1        588202002
2        588202003
3        588202004
4        588202005
           ...    
27352    240602017
27353    240602018
27354    240602019
27355    240602020
27356    240602021
Name: key, Length: 27357, dtype: object

In [89]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27941 entries, 0 to 27940
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회사명     27941 non-null  object 
 1   거래소코드   27941 non-null  int64  
 2   회계년도    27941 non-null  object 
 3   종가      21134 non-null  float64
 4   년       27941 non-null  int32  
 5   key     27941 non-null  object 
 6   종가변동률   27941 non-null  float64
dtypes: float64(2), int32(1), int64(1), object(3)
memory usage: 1.4+ MB


In [23]:
merged = pd.merge(df, price[['key','종가변동률']], on = 'key', how='outer')

In [25]:
merged

,회사명,거래소코드,회계년도,소속코드,상장폐지일,유동비율,당좌비율,유동부채비율,비유동비율,순운전자본비율,...,금융비용_대_부채비율,자기자본배율,총자산영업이익률,총자산회전율,log총자산,log총자산회전률,자본/부채,이익잉여금/총자산,key,종가변동률
0,(주)CMG제약,58820.0,2000/12,5.0,0,141.01,19.89,131.99,51.76,22.76,...,0.019074,179.37,0.052949,1.761228,10.274706,0.566011,0.801243,0.189100,588202001,-0.633803
1,(주)CMG제약,58820.0,2001/12,5.0,0,141.01,19.89,131.99,51.76,22.76,...,0.023517,179.37,0.014062,0.756792,9.947887,-0.278667,0.725225,0.005261,588202002,-0.633803
2,(주)CMG제약,58820.0,2002/12,5.0,0,114.21,16.60,132.37,84.50,7.98,...,0.043081,133.37,-0.247841,1.713785,9.642188,0.538704,0.737002,-0.139212,588202003,-0.633803
3,(주)CMG제약,58820.0,2003/12,5.0,0,185.63,23.55,71.46,39.31,35.58,...,0.058531,131.30,-0.082618,1.571816,9.311361,0.452232,1.389417,-0.203019,588202004,0.538462
4,(주)CMG제약,58820.0,2004/12,5.0,0,149.73,17.11,101.56,50.23,24.97,...,0.047346,52.25,-0.547982,2.526356,8.552367,0.926778,0.977472,-1.181502,588202005,-0.312500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,390102000,0.439286
28131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,261802000,-0.510309
28132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,536602000,-0.081967
28133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,320502000,9.421941


In [45]:
merged = merged.iloc[:27359,:] 
merged

,회사명,거래소코드,회계년도,소속코드,상장폐지일,유동비율,당좌비율,유동부채비율,비유동비율,순운전자본비율,...,금융비용_대_부채비율,자기자본배율,총자산영업이익률,총자산회전율,log총자산,log총자산회전률,자본/부채,이익잉여금/총자산,key,종가변동률
0,(주)CMG제약,58820.0,2000/12,5.0,0,141.01,19.89,131.99,51.76,22.76,...,0.019074,179.37,0.052949,1.761228,10.274706,0.566011,0.801243,0.189100,588202001,-0.633803
1,(주)CMG제약,58820.0,2001/12,5.0,0,141.01,19.89,131.99,51.76,22.76,...,0.023517,179.37,0.014062,0.756792,9.947887,-0.278667,0.725225,0.005261,588202002,-0.633803
2,(주)CMG제약,58820.0,2002/12,5.0,0,114.21,16.60,132.37,84.50,7.98,...,0.043081,133.37,-0.247841,1.713785,9.642188,0.538704,0.737002,-0.139212,588202003,-0.633803
3,(주)CMG제약,58820.0,2003/12,5.0,0,185.63,23.55,71.46,39.31,35.58,...,0.058531,131.30,-0.082618,1.571816,9.311361,0.452232,1.389417,-0.203019,588202004,0.538462
4,(주)CMG제약,58820.0,2004/12,5.0,0,149.73,17.11,101.56,50.23,24.97,...,0.047346,52.25,-0.547982,2.526356,8.552367,0.926778,0.977472,-1.181502,588202005,-0.312500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27354,흥구석유(주),24060.0,2016/12,5.0,0,411.65,341.54,5.19,93.01,14.15,...,0.012651,4611.84,0.012181,1.817620,11.278936,0.597528,6.945582,0.414800,240602017,0.126645
27355,흥구석유(주),24060.0,2017/12,5.0,0,177.50,146.69,12.83,91.80,8.68,...,0.012502,4650.29,0.009398,1.946103,11.288681,0.665829,6.866903,0.417422,240602018,-0.189781
27356,흥구석유(주),24060.0,2018/12,5.0,0,329.17,290.09,5.40,89.00,11.59,...,0.018056,4696.15,0.014520,2.210036,11.227986,0.793009,14.789418,0.453302,240602019,-0.025225
27357,흥구석유(주),24060.0,2019/12,5.0,0,485.39,422.76,6.05,79.27,21.47,...,0.000300,5133.42,0.005988,1.800947,11.334528,0.588312,11.561712,0.485889,240602020,0.981516


In [28]:
for i in merged:
    print(merged.isna().sum()[i])

776
776
776
776
776
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
837
1484
919
935
919
919
919
930
935
935
944
1954
987
1686
1493
1487
2749
1484
1484
1484
837
919
837
919
919
919
935
919
919
0
0


In [22]:
for i in merged.columns:
    print(merged[i].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


## 데이터 저장

In [44]:
merged.to_csv('dataset+주가변동률_.csv')